In [221]:
import numpy as np
from numpy import array,hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [222]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [223]:
# define input sequence
a = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
b = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([a[i]+b[i] for i in range(len(a))])
out_seq.shape

(9,)

In [224]:
# convert to [rows, columns] structure
a = a.reshape((len(a), 1))
b = b.reshape((len(b), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

In [225]:
out_seq.shape

(9, 1)

In [288]:
dataset = hstack((a, b, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [289]:
dataset.shape

(9, 3)

In [292]:
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

In [293]:
X,y

(array([[[10, 15],
         [20, 25],
         [30, 35]],
 
        [[20, 25],
         [30, 35],
         [40, 45]],
 
        [[30, 35],
         [40, 45],
         [50, 55]],
 
        [[40, 45],
         [50, 55],
         [60, 65]],
 
        [[50, 55],
         [60, 65],
         [70, 75]],
 
        [[60, 65],
         [70, 75],
         [80, 85]],
 
        [[70, 75],
         [80, 85],
         [90, 95]]]),
 array([ 65,  85, 105, 125, 145, 165, 185]))

In [294]:
X.shape,y.shape

((7, 3, 2), (7,))

In [295]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)

Epoch 1/300
1/1 [==============================] - 2s 2s/step - loss: 17771.4492
Epoch 2/300
1/1 [==============================] - 0s 14ms/step - loss: 17540.1602
Epoch 3/300
1/1 [==============================] - 0s 13ms/step - loss: 17315.9824
Epoch 4/300
1/1 [==============================] - 0s 17ms/step - loss: 17097.0059
Epoch 5/300
1/1 [==============================] - 0s 15ms/step - loss: 16887.6465
Epoch 6/300
1/1 [==============================] - 0s 15ms/step - loss: 16686.6758
Epoch 7/300
1/1 [==============================] - 0s 12ms/step - loss: 16494.5078
Epoch 8/300
1/1 [==============================] - 0s 12ms/step - loss: 16308.0469
Epoch 9/300
1/1 [==============================] - 0s 14ms/step - loss: 16130.9609
Epoch 10/300
1/1 [==============================] - 0s 12ms/step - loss: 15965.4404
Epoch 11/300
1/1 [==============================] - 0s 10ms/step - loss: 15798.2207
Epoch 12/300
1/1 [==============================] - 0s 14ms/step - loss: 15610.0439
Epo

1/1 [==============================] - 0s 12ms/step - loss: 0.3791
Epoch 203/300
1/1 [==============================] - 0s 11ms/step - loss: 0.3737
Epoch 204/300
1/1 [==============================] - 0s 12ms/step - loss: 0.3684
Epoch 205/300
1/1 [==============================] - 0s 17ms/step - loss: 0.3631
Epoch 206/300
1/1 [==============================] - 0s 14ms/step - loss: 0.3577
Epoch 207/300
1/1 [==============================] - 0s 9ms/step - loss: 0.3525
Epoch 208/300
1/1 [==============================] - 0s 18ms/step - loss: 0.3473
Epoch 209/300
1/1 [==============================] - 0s 15ms/step - loss: 0.3421
Epoch 210/300
1/1 [==============================] - 0s 13ms/step - loss: 0.3369
Epoch 211/300
1/1 [==============================] - 0s 19ms/step - loss: 0.3318
Epoch 212/300
1/1 [==============================] - 0s 14ms/step - loss: 0.3267
Epoch 213/300
1/1 [==============================] - 0s 17ms/step - loss: 0.3217
Epoch 214/300
1/1 [========================

In [296]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 3, 50)             10600     
                                                                 
 lstm_22 (LSTM)              (None, 50)                20200     
                                                                 
 dense_12 (Dense)            (None, 1)                 51        
                                                                 
Total params: 30,851
Trainable params: 30,851
Non-trainable params: 0
_________________________________________________________________


In [317]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
temp_input=list(x_input)
lst_output=[]
i=0

In [318]:
x_input

array([[ 80,  85],
       [ 90,  95],
       [100, 105]])

In [319]:
x_input = x_input.reshape((1, n_steps, n_features))
model.predict(x_input, verbose=0)

array([[206.16783]], dtype=float32)

In [1]:
# while(i<50):
#     if(len(temp_input)>3):
#         x_input=array(temp_input[1:])
#         print("{} day input {}".format(i,x_input))
#         x_input = x_input.reshape((1, n_steps, n_features))
#         yhat = model.predict(x_input, verbose=0)
#         print("{} day output {}".format(i,yhat))
#         temp_input.append(yhat[0][0])
#         temp_input=temp_input[1:]
#         lst_output.append(yhat[0][0])
#         i=i+1
#     else:
#         x_input = x_input.reshape((1, n_steps, n_features))
#         yhat = model.predict(x_input, verbose=0)
#         print(yhat[0])
#         temp_input.append(yhat[0][0])
#         lst_output.append(yhat[0][0])
#         i=i+1
# print(lst_output)
